Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [297]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [298]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [299]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [300]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [301]:
#importing everything I need
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

In [302]:
#Data Wrangling

def wrangle(filepath):
  df = pd.read_csv(filepath,
                   parse_dates=[20],
                   dayfirst=True,
                   index_col=20).sort_index()
  df = df[df['BUILDING CLASS CATEGORY'].str.contains('01 ONE FAMILY DWELLINGS')]      
  df['SALE PRICE'] = (
    df['SALE PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
    )
  df = df[df['SALE PRICE'] >= 100000]
  df = df[df['SALE PRICE'] <= 2000000]
  #Removing high cardinality and NAN columns
  cutoff = 15
  
  drop_cols = [col for col in df.select_dtypes('object')
               if df[col].nunique() >= cutoff]
  df.drop(columns=drop_cols, inplace=True)
  df.drop(['EASE-MENT'], axis = 1, inplace=True)
  df.drop(['APARTMENT NUMBER'], axis = 1, inplace=True)
  df.drop(['BLOCK'], axis = 1, inplace=True)
  df.drop(['LOT'], axis = 1, inplace=True)
  df.drop(['ZIP CODE'], axis = 1, inplace=True)
  

  return df


    
df = wrangle(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

In [303]:
df.head()

,BOROUGH,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
SALE DATE,,,,,,,,,,,,
2019-01-01,3,01 ONE FAMILY DWELLINGS,1,A9,1.0,0.0,1.0,1325.0,1930.0,1,A9,550000
2019-01-01,4,01 ONE FAMILY DWELLINGS,1,A1,1.0,0.0,1.0,2001.0,1940.0,1,A1,200000
2019-01-02,2,01 ONE FAMILY DWELLINGS,1,A5,1.0,0.0,1.0,1842.0,1940.0,1,A5,578000
2019-01-02,2,01 ONE FAMILY DWELLINGS,1,A1,1.0,0.0,1.0,1080.0,1915.0,1,A1,430000
2019-01-02,3,01 ONE FAMILY DWELLINGS,1,S1,1.0,1.0,2.0,2800.0,1933.0,1,S1,1330000


In [304]:
#Splitting data

target = 'SALE PRICE'
y = df[target]
X = df.drop(columns=target)

#Splitting training and test sets

cutoff = '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]


In [305]:
#Establishing Baseline 

y_pred = [y_train.mean()] * len(y_train)
print('Mean price', y_train.mean())
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Mean price 617953.2514405448
Baseline MAE: 212016.29324193168


In [306]:
#OHE for categorical variables

ohe = OneHotEncoder(use_cat_names=True)

ohe.fit(X_train)

XT_train = ohe.transform(X_train)
XT_test = ohe.transform(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
#Model for Linear Regression
model_lr = LinearRegression()
model_lr.fit(XT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#SelectKBest model Linear Regression

skb = SelectKBest(f_regression, k=15)

skb.fit(XT_train, y_train)

XTT_train = skb.transform(XT_train)
XTT_test = skb.transform(XT_test)

model_lr2 = LinearRegression()
model_lr2.fit(XTT_train, y_train)

In [ ]:
#Model for Ridge regression
model_r = Ridge(alpha=1.0,normalize=True) 
model_r.fit(XT_train, y_train)

In [318]:

print('LR test MAE:', mean_absolute_error(y_test, model_lr.predict(XT_test)))

LR test MAE: 195164.96020224577


In [319]:

print('SKLR test MAE:', mean_absolute_error(y_test, model_lr2.predict(XTT_test)))

SKLR test MAE: 196842.16766855013


In [320]:

print('Ridge test MAE:', mean_absolute_error(y_test, model_r.predict(XT_test)))

Ridge test MAE: 200963.6454687995
